In [ ]:
from transformers import SiglipImageProcessor, SiglipVisionModel,SiglipVisionConfig
from PIL import Image


In [ ]:
import torch
import torch.nn as nn
torch.manual_seed(42)
strength=0.3

vision_tower_hidden_dim=8
vision_tower_name='vq'

# 设置缩放因子范围（可以根据需要调整）
scale_min, scale_max = 1+strength, 1-strength
shear_std=1*strength
# 生成两个随机缩放因子
scale_factors = torch.rand(vision_tower_hidden_dim) * (scale_max - scale_min) + scale_min  # 范围 [scale_min, scale_max]
# 创建一个对角矩阵，缩放因子在对角线上
scale_matrix = torch.diag(scale_factors)
#shear = torch.eye(vision_tower_hidden_dim)
# 生成上三角部分的随机值（非对角线元素）
# upper_triangle = torch.triu(
#     torch.rand(vision_tower_hidden_dim, vision_tower_hidden_dim) * shear_std,
#     diagonal=1  # 仅填充严格上三角部分
# )
# shear = shear + upper_triangle

# 3. 组合缩放和剪切（顺序：先缩放，后剪切）
#scale_matrix = shear @ scale_matrix
torch.save(scale_matrix, f"{vision_tower_name}_{strength}_scale_matrix.pt")

random_matrix = torch.rand(vision_tower_hidden_dim, vision_tower_hidden_dim)
# QR 分解得到正交矩阵 Q
rotation_matrix, _ = torch.linalg.qr(random_matrix, mode='complete')
torch.save(rotation_matrix, f"{vision_tower_name}_{strength}_rotation_matrix.pt")

translation_matrix=torch.rand(vision_tower_hidden_dim) * (scale_max - scale_min) + scale_min-1
torch.save(translation_matrix, f"{vision_tower_name}_{strength}_translation_matrix.pt")



In [ ]:
import torch
import torch.nn as nn
torch.manual_seed(42)


class Affine(nn.Module):
    def __init__(self, dim,strength,vision_tower_name):
        super(Affine, self).__init__()
        
        Q= torch.load(f"{vision_tower_name}_{strength}_rotation_matrix.pt")
        S=torch.load(f"{vision_tower_name}_{strength}_scale_matrix.pt")
        Q=torch.matmul(Q,S)
        B=torch.load(f"{vision_tower_name}_{strength}_translation_matrix.pt")
        # 将 A 和 b 作为常量属性存储，并确保它们不会被训练
        self.register_buffer('A', Q)  # register_buffer 确保 A 不可训练
        self.register_buffer('b', B)  # 随机平移向量 b，不可训练

    def forward(self, x, *args, **kwargs):
        pass

In [ ]:

transformation= Affine(vision_tower_hidden_dim, strength, vision_tower_name)

torch.save(transformation.state_dict(), f"{vision_tower_name}affine_{strength}.pth")